In [1]:
# Pagrindinės bibliotekos
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns  # Vizualizacijai, atkreipkite dėmesį, kad naudojame "matplotlib" pagrindu
import warnings
import requests
from datetime import datetime
from bs4 import BeautifulSoup
import time
import sqlite3

# Duomenų modeliavimas ir analizė
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler

# Darbas su interneto naršyklėmis per Selenium
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

# Įspėjimų ignoravimas
warnings.filterwarnings('ignore')

# Specializuoti moduliai
from numpy.polynomial import Polynomial

import json
from sklearn.preprocessing import StandardScaler
from datetime import timedelta

In [2]:
HistoricalWildfires = pd.read_csv('C:/Users/Pauliussl/Paskaita89/Historical_Wildfires.csv')
HistoricalWeather = pd.read_csv('C:/Users/Pauliussl/Paskaita89/HistoricalWeather.csv')
HistoricalWeatherForecasts = pd.read_csv('C:/Users/Pauliussl/Paskaita89/HistoricalWeatherForecasts.csv')
LandClass = pd.read_csv('C:/Users/Pauliussl/Paskaita89/LandClass.csv')
VegetationIndex = pd.read_csv('C:/Users/Pauliussl/Paskaita89/VegetationIndex.csv')


In [3]:
HistoricalWildfires
HistoricalWildfires['Date'] = pd.to_datetime(HistoricalWildfires['Date']).dt.strftime('%Y-%m-%d')
Wildfires_LandClass = pd.merge(HistoricalWildfires, LandClass, on='Region', how='left')
Wildfires_LandClass

,Region,Date,Estimated_fire_area,Mean_estimated_fire_brightness,Mean_estimated_fire_radiative_power,Mean_confidence,Std_confidence,Var_confidence,Count,Replaced,...,Bare / sparse vegetation,Permanent water bodies,Herbaceous wetland,"Closed forest, evergreen, broad leaf","Closed forest, deciduous broad leaf","Closed forest, unknown","Open forest, evergreen broad leaf","Open forest, deciduous broad leaf","Open forest, unknown definitions",Open sea
0,NSW,2005-01-04,8.680000,312.266667,42.400000,78.666667,2.886751,8.333333,3,R,...,0.2,0.2,0.1,14.7,6.8,0.3,0.5,3.7,10.3,0.1
1,NSW,2005-01-05,16.611250,322.475000,62.362500,85.500000,8.088793,65.428571,8,R,...,0.2,0.2,0.1,14.7,6.8,0.3,0.5,3.7,10.3,0.1
2,NSW,2005-01-06,5.520000,325.266667,38.400000,78.333333,3.214550,10.333333,3,R,...,0.2,0.2,0.1,14.7,6.8,0.3,0.5,3.7,10.3,0.1
3,NSW,2005-01-07,6.264000,313.870000,33.800000,92.200000,7.529940,56.700000,5,R,...,0.2,0.2,0.1,14.7,6.8,0.3,0.5,3.7,10.3,0.1
4,NSW,2005-01-08,5.400000,337.383333,122.533333,91.000000,7.937254,63.000000,3,R,...,0.2,0.2,0.1,14.7,6.8,0.3,0.5,3.7,10.3,0.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26401,WA,2020-10-27,32.970000,324.068182,103.318182,84.363636,5.277741,27.854545,11,N,...,1.0,0.4,0.0,1.2,2.4,0.2,0.1,4.8,8.7,0.8
26402,WA,2020-10-28,20.840625,314.425000,37.256250,88.312500,8.121730,65.962500,16,N,...,1.0,0.4,0.0,1.2,2.4,0.2,0.1,4.8,8.7,0.8
26403,WA,2020-10-29,136.083077,323.588461,294.102564,92.974359,8.282789,68.604588,39,N,...,1.0,0.4,0.0,1.2,2.4,0.2,0.1,4.8,8.7,0.8
26404,WA,2020-10-30,42.397895,324.205263,54.994737,89.631579,8.461107,71.590327,38,N,...,1.0,0.4,0.0,1.2,2.4,0.2,0.1,4.8,8.7,0.8


In [4]:
weather_pivot = HistoricalWeather.pivot_table(
    index=['Date', 'Region'],          # Grupavimo pagrindas
    columns='Parameter',               # Kiekvienas unikalus parametras taps atskiru stulpeliu
    values='mean()',                   # Šiame pavyzdyje 'mean()' reikšmė yra teisinga, remiantis naujausia ekrano nuotrauka
    aggfunc='first'                    # Naudojame 'first', nes norime paimti pirmą reikšmę, jei jų yra daugiau nei viena
)

weather_pivot.reset_index(inplace=True)
weather_pivot.columns = ['_'.join(col).strip() if type(col) is tuple else col for col in weather_pivot.columns]    # Pakeiskite stulpelių pavadinimus, kad atspindėtų originalius parametrus
weather_pivot


,Date,Region,Precipitation,RelativeHumidity,SoilWaterContent,SolarRadiation,Temperature,WindSpeed
0,2005-01-01,NSW,0.044274,36.355567,0.170931,26.749389,27.341182,3.323550
1,2005-01-01,NT,9.884958,61.494675,0.167735,19.781791,29.881492,5.296892
2,2005-01-01,QL,1.453053,47.959364,0.185641,27.056979,28.842866,3.483753
3,2005-01-01,SA,0.059078,30.057683,0.056047,27.142643,30.793675,4.657538
4,2005-01-01,TA,3.099497,65.086764,0.211360,26.755711,11.788805,5.408138
...,...,...,...,...,...,...,...,...
40476,2020-10-31,QL,0.386800,35.829668,0.141467,28.745128,24.777835,3.137745
40477,2020-10-31,SA,0.006940,45.700218,0.083082,26.530568,18.947783,3.548225
40478,2020-10-31,TA,0.472709,77.645878,0.264608,21.782732,11.648813,2.501697
40479,2020-10-31,VI,0.822028,78.326366,0.324260,19.553751,13.167147,3.838360


In [5]:
final_data = pd.merge(Wildfires_LandClass, weather_pivot, on=['Region', 'Date'], how='left')
final_data


,Region,Date,Estimated_fire_area,Mean_estimated_fire_brightness,Mean_estimated_fire_radiative_power,Mean_confidence,Std_confidence,Var_confidence,Count,Replaced,...,"Open forest, evergreen broad leaf","Open forest, deciduous broad leaf","Open forest, unknown definitions",Open sea,Precipitation,RelativeHumidity,SoilWaterContent,SolarRadiation,Temperature,WindSpeed
0,NSW,2005-01-04,8.680000,312.266667,42.400000,78.666667,2.886751,8.333333,3,R,...,0.5,3.7,10.3,0.1,2.808620,57.095628,0.214293,22.617291,23.055527,4.841764
1,NSW,2005-01-05,16.611250,322.475000,62.362500,85.500000,8.088793,65.428571,8,R,...,0.5,3.7,10.3,0.1,0.157935,47.170735,0.203951,28.076835,22.425765,4.014080
2,NSW,2005-01-06,5.520000,325.266667,38.400000,78.333333,3.214550,10.333333,3,R,...,0.5,3.7,10.3,0.1,0.434833,39.874638,0.193668,30.561703,20.621405,4.477879
3,NSW,2005-01-07,6.264000,313.870000,33.800000,92.200000,7.529940,56.700000,5,R,...,0.5,3.7,10.3,0.1,0.064224,39.279410,0.183778,32.143718,19.632722,4.017578
4,NSW,2005-01-08,5.400000,337.383333,122.533333,91.000000,7.937254,63.000000,3,R,...,0.5,3.7,10.3,0.1,0.203352,42.400824,0.175935,29.347715,21.477315,3.884550
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26401,WA,2020-10-27,32.970000,324.068182,103.318182,84.363636,5.277741,27.854545,11,N,...,0.1,4.8,8.7,0.8,0.123625,33.320530,0.046453,27.702953,27.784354,4.268756
26402,WA,2020-10-28,20.840625,314.425000,37.256250,88.312500,8.121730,65.962500,16,N,...,0.1,4.8,8.7,0.8,0.196366,38.782971,0.046638,26.916920,27.351996,4.365700
26403,WA,2020-10-29,136.083077,323.588461,294.102564,92.974359,8.282789,68.604588,39,N,...,0.1,4.8,8.7,0.8,0.188249,36.019309,0.047551,26.880282,26.532585,4.496722
26404,WA,2020-10-30,42.397895,324.205263,54.994737,89.631579,8.461107,71.590327,38,N,...,0.1,4.8,8.7,0.8,0.334691,33.523594,0.051021,25.992187,26.231115,4.211237


In [6]:
final_data2 = final_data.select_dtypes(include=[np.number])
correlation = final_data2.corr()['Estimated_fire_area'].sort_values()
correlation

RelativeHumidity                                           -0.343011
SoilWaterContent                                           -0.301357
Closed forest, evergreen, broad leaf                       -0.196779
Urban / built up                                           -0.182718
Cultivated and managed vegetation/agriculture (cropland)   -0.170654
Closed forest, unknown                                     -0.150895
Permanent water bodies                                     -0.146306
Open forest, evergreen broad leaf                          -0.133206
Herbaceous wetland                                         -0.127265
Precipitation                                              -0.120846
Bare / sparse vegetation                                   -0.044826
Open sea                                                   -0.043477
Closed forest, deciduous broad leaf                         0.032731
Open forest, unknown definitions                            0.084551
Var_confidence                    

In [7]:
final_data

,Region,Date,Estimated_fire_area,Mean_estimated_fire_brightness,Mean_estimated_fire_radiative_power,Mean_confidence,Std_confidence,Var_confidence,Count,Replaced,...,"Open forest, evergreen broad leaf","Open forest, deciduous broad leaf","Open forest, unknown definitions",Open sea,Precipitation,RelativeHumidity,SoilWaterContent,SolarRadiation,Temperature,WindSpeed
0,NSW,2005-01-04,8.680000,312.266667,42.400000,78.666667,2.886751,8.333333,3,R,...,0.5,3.7,10.3,0.1,2.808620,57.095628,0.214293,22.617291,23.055527,4.841764
1,NSW,2005-01-05,16.611250,322.475000,62.362500,85.500000,8.088793,65.428571,8,R,...,0.5,3.7,10.3,0.1,0.157935,47.170735,0.203951,28.076835,22.425765,4.014080
2,NSW,2005-01-06,5.520000,325.266667,38.400000,78.333333,3.214550,10.333333,3,R,...,0.5,3.7,10.3,0.1,0.434833,39.874638,0.193668,30.561703,20.621405,4.477879
3,NSW,2005-01-07,6.264000,313.870000,33.800000,92.200000,7.529940,56.700000,5,R,...,0.5,3.7,10.3,0.1,0.064224,39.279410,0.183778,32.143718,19.632722,4.017578
4,NSW,2005-01-08,5.400000,337.383333,122.533333,91.000000,7.937254,63.000000,3,R,...,0.5,3.7,10.3,0.1,0.203352,42.400824,0.175935,29.347715,21.477315,3.884550
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26401,WA,2020-10-27,32.970000,324.068182,103.318182,84.363636,5.277741,27.854545,11,N,...,0.1,4.8,8.7,0.8,0.123625,33.320530,0.046453,27.702953,27.784354,4.268756
26402,WA,2020-10-28,20.840625,314.425000,37.256250,88.312500,8.121730,65.962500,16,N,...,0.1,4.8,8.7,0.8,0.196366,38.782971,0.046638,26.916920,27.351996,4.365700
26403,WA,2020-10-29,136.083077,323.588461,294.102564,92.974359,8.282789,68.604588,39,N,...,0.1,4.8,8.7,0.8,0.188249,36.019309,0.047551,26.880282,26.532585,4.496722
26404,WA,2020-10-30,42.397895,324.205263,54.994737,89.631579,8.461107,71.590327,38,N,...,0.1,4.8,8.7,0.8,0.334691,33.523594,0.051021,25.992187,26.231115,4.211237


In [8]:
top_influencing_columns = ["Count", "Mean_estimated_fire_brightness", "Mean_confidence", "SolarRadiation", "Std_confidence", "WindSpeed", "Temperature", "Shrubs", "Herbaceous_vegetation", "Var_confidence"]
ML_data = final_data[["Estimated_fire_area", "Date", "Count", "Mean_estimated_fire_brightness", "Mean_confidence", "SolarRadiation", "Std_confidence", "WindSpeed", "Temperature", "Shrubs", "Herbaceous vegetation", "Var_confidence"]]
ML_data




,Estimated_fire_area,Date,Count,Mean_estimated_fire_brightness,Mean_confidence,SolarRadiation,Std_confidence,WindSpeed,Temperature,Shrubs,Herbaceous vegetation,Var_confidence
0,8.680000,2005-01-04,3,312.266667,78.666667,22.617291,2.886751,4.841764,23.055527,6.2,43.6,8.333333
1,16.611250,2005-01-05,8,322.475000,85.500000,28.076835,8.088793,4.014080,22.425765,6.2,43.6,65.428571
2,5.520000,2005-01-06,3,325.266667,78.333333,30.561703,3.214550,4.477879,20.621405,6.2,43.6,10.333333
3,6.264000,2005-01-07,5,313.870000,92.200000,32.143718,7.529940,4.017578,19.632722,6.2,43.6,56.700000
4,5.400000,2005-01-08,3,337.383333,91.000000,29.347715,7.937254,3.884550,21.477315,6.2,43.6,63.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
26401,32.970000,2020-10-27,11,324.068182,84.363636,27.702953,5.277741,4.268756,27.784354,31.3,43.5,27.854545
26402,20.840625,2020-10-28,16,314.425000,88.312500,26.916920,8.121730,4.365700,27.351996,31.3,43.5,65.962500
26403,136.083077,2020-10-29,39,323.588461,92.974359,26.880282,8.282789,4.496722,26.532585,31.3,43.5,68.604588
26404,42.397895,2020-10-30,38,324.205263,89.631579,25.992187,8.461107,4.211237,26.231115,31.3,43.5,71.590327


In [9]:
ML_data['Date'] = pd.to_datetime(ML_data['Date'])
ML_data['Month'] = ML_data['Date'].dt.month
ML_data['Day'] = ML_data['Date'].dt.day
ML_data.drop('Date', axis=1, inplace=True)
ML_data

,Estimated_fire_area,Count,Mean_estimated_fire_brightness,Mean_confidence,SolarRadiation,Std_confidence,WindSpeed,Temperature,Shrubs,Herbaceous vegetation,Var_confidence,Month,Day
0,8.680000,3,312.266667,78.666667,22.617291,2.886751,4.841764,23.055527,6.2,43.6,8.333333,1,4
1,16.611250,8,322.475000,85.500000,28.076835,8.088793,4.014080,22.425765,6.2,43.6,65.428571,1,5
2,5.520000,3,325.266667,78.333333,30.561703,3.214550,4.477879,20.621405,6.2,43.6,10.333333,1,6
3,6.264000,5,313.870000,92.200000,32.143718,7.529940,4.017578,19.632722,6.2,43.6,56.700000,1,7
4,5.400000,3,337.383333,91.000000,29.347715,7.937254,3.884550,21.477315,6.2,43.6,63.000000,1,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...
26401,32.970000,11,324.068182,84.363636,27.702953,5.277741,4.268756,27.784354,31.3,43.5,27.854545,10,27
26402,20.840625,16,314.425000,88.312500,26.916920,8.121730,4.365700,27.351996,31.3,43.5,65.962500,10,28
26403,136.083077,39,323.588461,92.974359,26.880282,8.282789,4.496722,26.532585,31.3,43.5,68.604588,10,29
26404,42.397895,38,324.205263,89.631579,25.992187,8.461107,4.211237,26.231115,31.3,43.5,71.590327,10,30


In [10]:
ML_data.to_csv("ML_data.csv", index=False)



In [11]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

data = ML_data
X = data.drop('Estimated_fire_area', axis=1)
y = data['Estimated_fire_area']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR

# Load your dataset
data = ML_data

# Fill missing values with the mean of each column
for column in data.columns:
    data[column].fillna(data[column].mean(), inplace=True)

# Selecting the target variable and predictors
X = data.drop('Estimated_fire_area', axis=1)
y = data['Estimated_fire_area']

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Atsitiktinių Miškų Regresorius
model = RandomForestRegressor(random_state=42)
model.fit(X_train, y_train)

# Linijinė Regresija
model_linear = LinearRegression()
model_linear.fit(X_train, y_train)

# Polinominė Regresija
model_poly = make_pipeline(PolynomialFeatures(degree=2), LinearRegression())
model_poly.fit(X_train, y_train)

# Sprendimų Medžio Regresorius
model_dtree_reg = DecisionTreeRegressor(random_state=42)
model_dtree_reg.fit(X_train, y_train)

# Atsitiktinių Miškų Regresorius
model_random_forest = RandomForestRegressor(random_state=42)
model_random_forest.fit(X_train, y_train)

# Paramos Vektorių Mašina (regresija)
model_svr = SVR()
model_svr.fit(X_train, y_train)




SVR()

In [13]:
from sklearn.metrics import r2_score

# Apskaičiuojame R² įverčius testavimo imčiai
r2_random_forest = r2_score(y_test, model_random_forest.predict(X_test))
r2_linear = r2_score(y_test, model_linear.predict(X_test))
r2_poly = r2_score(y_test, model_poly.predict(X_test))
r2_dtree_reg = r2_score(y_test, model_dtree_reg.predict(X_test))
r2_svr = r2_score(y_test, model_svr.predict(X_test))

# Atspausdiname R² įverčius
print("Random Forest R² Score: ", r2_random_forest)
print("Linear Regression R² Score: ", r2_linear)
print("Polynomial Regression R² Score: ", r2_poly)
print("Decision Tree Regressor R² Score: ", r2_dtree_reg)
print("Support Vector Regression R² Score: ", r2_svr)



Random Forest R² Score:  0.9468801712001949
Linear Regression R² Score:  0.9424940751925116
Polynomial Regression R² Score:  0.947125439602388
Decision Tree Regressor R² Score:  0.9052602869250379
Support Vector Regression R² Score:  0.553858379028227


In [21]:
Faktas_spejimas = final_data[['Date', 'Estimated_fire_area']]
Faktas_spejimas['Modelio_spėjimas'] = pd.NA
Faktas_spejimas['Date'] = pd.to_datetime(Faktas_spejimas['Date'])
Faktas_spejimas['Year'] = Faktas_spejimas['Date'].dt.year
Faktas_spejimas['Month'] = Faktas_spejimas['Date'].dt.month
Faktas_spejimas['Day'] = Faktas_spejimas['Date'].dt.day
Faktas_spejimas.drop('Date', axis=1, inplace=True)
Faktas_spejimas

,Estimated_fire_area,Modelio_spėjimas,Year,Month,Day
0,8.680000,<NA>,2005,1,4
1,16.611250,<NA>,2005,1,5
2,5.520000,<NA>,2005,1,6
3,6.264000,<NA>,2005,1,7
4,5.400000,<NA>,2005,1,8
...,...,...,...,...,...
26401,32.970000,<NA>,2020,10,27
26402,20.840625,<NA>,2020,10,28
26403,136.083077,<NA>,2020,10,29
26404,42.397895,<NA>,2020,10,30
